In [48]:
# use ansible to let argocd setup platform (minio, postgres, mlflow)

In [32]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
export ANSIBLE_CONFIG=/work/MLSysEngOps-NYU25Spring/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles

In [29]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_platform.yml


PLAY [Mount persistent volume and deploy MLflow platform with MinIO] ***********

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Detect filesystem on /dev/vdb] *******************************************
ok: [node1]

TASK [Format /dev/vdb as ext4 on first use] ************************************
skipping: [node1]

TASK [Make root mount directory] ***********************************************
ok: [node1]

TASK [Mount volume & ensure /etc/fstab] ****************************************
ok: [node1]

TASK [Make MinIO data dir] *****************************************************
ok: [node1]

TASK [Make Postgres data dir] **************************************************
ok: [node1]

PLAY [Deploy MLflow platform via ArgoCD & Helm with MinIO secret handling] *****

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password from Kubernetes secret] *********************

In [40]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/workflow_build_init.yml


PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update gourmetgram-iac repo] ************************************
changed: [node1]

TASK [Submit Argo Workflow] ****************************************************
changed: [node1]

TASK [Extract Workflow Name] ***************************************************
ok: [node1]

TASK [Wait for workflow to complete (success or fail)] *************************
changed: [node1]

TASK [Get final workflow result] ***********************************************
changed: [node1]

TASK [Display workflow phase] **************************************************
ok: [node1] => 
  msg: 'Workflow kaniko-pvc-build-9xghg finished with status: Succeeded'

TASK [Fail if workflow did not succeed] ****************************************
skipping: [node1]

PLAY RECAP ***************************************************

In [42]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_staging.yml


PLAY [Deploy Project37 Staging via ArgoCD & Helm] ******************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create project37-staging namespace if missing] ***************************
ok: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
skipping: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
changed: [node1]

TASK [Display ArgoCD app status] *****************

In [59]:
cd /work/MLSysEngOps-NYU25Spring/ansible
cat argocd/argocd_add_canary2.yml

---
- name: Deploy Project37 Canary via ArgoCD & Helm
  hosts: node1
  become: yes
  vars:
    app_name: project37-canary
    repo_url: https://github.com/zuchuandatou/MLSysEngOps-NYU25Spring.git
    target_rev: zehao-li/cicd
    app_path: k8s/canary
    argocd_namespace: argocd
    dest_namespace: project37-canary

  tasks:

    - name: Get ArgoCD admin password
      command: >
        kubectl get secret -n {{ argocd_namespace }} argocd-initial-admin-secret \
        -o jsonpath="{.data.password}"
      register: argocd_password_b64

    - name: Decode ArgoCD password
      shell: echo {{ argocd_password_b64.stdout }} | base64 --decode
      register: argocd_password

    - name: Login to ArgoCD
      command: >
        argocd login localhost --username admin \
        --password "{{ argocd_password.stdout }}" \
        --grpc-web --port-forward --port-forward-namespace={{ argocd_namespace }}
      changed_when: false

    - name: Detect external IP starting with 10.56
      set_fact

In [60]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_canary2.yml


PLAY [Deploy Project37 Canary via ArgoCD & Helm] *******************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create project37-canary namespace if missing] ****************************
ok: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] *****************

In [44]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_prod.yml


PLAY [Deploy Project37 Production via ArgoCD & Helm] ***************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create gourmetgram-production namespace if missing] **********************
changed: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] ************